In [1]:
import argparse
import datetime
import glob
import math
import numpy as np
import os
import pandas as pd
import rasterio
import seaborn as sns
import statsmodels.api as sm
import time
import xarray as xr

import sys
sys.path.append('../')

from functools import reduce
from matplotlib import pyplot as plt
from scipy.interpolate import interp1d

from utilsTX import power_curve
from utilsTX import windpower_simulation_era5


from dask.diagnostics import ProgressBar
ProgressBar().register()

from paths_usa import *

In [2]:
data_path = '/data/users/kgruber/other-data/TX/'
gwacf_path = results_path
results_path = '/data/users/kgruber/results/TX/'

In [3]:
GWA = '2'

# Simulate wind power with ERA5
wind1 = xr.open_mfdataset(era_path + "/eff_ws/era5_wind_USA_*.nc", chunks = {'time': 38})
wind = wind1.sortby('time')
alpha1 = xr.open_mfdataset(era_path + "/eff_ws/era5_alpha_USA_*.nc", chunks = {'time': 38})
alpha = alpha1.sortby('time')

# with GWA
outfile = results_path + '/windpower_TX_ERA5_GWA*.nc'
turbine_data_era_gwa = pd.read_csv(data_path + '/turbine_data_era_gwa' + GWA + '.csv', parse_dates=['commissioning'])

In [32]:
for y in range(0,70,5):
    year1 = str(1950+y)
    year2 = str(1950+y+4)
    if results_path + 'windpower_TX_ERA5_GWA_' + year1 + '-' + year2 + '.nc' not in glob.glob(outfile):
        print('calculating ERA5 TX GWA ' + year1 + '-' + year2)
        GWAcf = xr.open_dataarray(gwacf_path + '/cf_ERA_GWA2.nc')
        wps = windpower_simulation_era5(wind.wh100.sel(time=slice(year1,year2)),
                                        alpha.alpha.sel(time=slice(year1,year2)),
                                        turbine_data_era_gwa.height.values,
                                        turbine_data_era_gwa.capacity.values,
                                        turbine_data_era_gwa.sp.values,
                                        turbine_data_era_gwa.lon.values,
                                        turbine_data_era_gwa.lat.values,
                                        GWAcf)
        # save as netcdf
        wps.to_dataset(name='wp').to_netcdf(results_path+"/windpower_TX_ERA5_GWA_" + year1 + "-" + year2 + ".nc")

calculating ERA5 TX GWA 1985-1989
[########################################] | 100% Completed | 45.7s
[########################################] | 100% Completed | 52.8s
calculating ERA5 TX GWA 1990-1994
[########################################] | 100% Completed | 52.0s
[########################################] | 100% Completed | 55.0s
calculating ERA5 TX GWA 1995-1999
[########################################] | 100% Completed |  1min  4.5s
[########################################] | 100% Completed | 48.2s
calculating ERA5 TX GWA 2000-2004
[########################################] | 100% Completed |  1min  4.3s
[########################################] | 100% Completed | 48.5s
calculating ERA5 TX GWA 2005-2009
[########################################] | 100% Completed |  1min  2.9s
[########################################] | 100% Completed | 49.4s
calculating ERA5 TX GWA 2010-2014
[########################################] | 100% Completed |  1min  7.9s
[#######################

In [4]:
# for 2020 and 2021
year1 = '2020'
year2 = '2021'
if results_path + 'windpower_TX_ERA5_GWA_' + year1 + '-' + year2 + '.nc' not in glob.glob(outfile):
    print('calculating ERA5 TX GWA ' + year1 + '-' + year2)
    GWAcf = xr.open_dataarray(gwacf_path + '/cf_ERA_GWA2.nc')
    wps = windpower_simulation_era5(wind.wh100.sel(time=slice(year1,year2)),
                                    alpha.alpha.sel(time=slice(year1,year2)),
                                    turbine_data_era_gwa.height.values,
                                    turbine_data_era_gwa.capacity.values,
                                    turbine_data_era_gwa.sp.values,
                                    turbine_data_era_gwa.lon.values,
                                    turbine_data_era_gwa.lat.values,
                                    GWAcf)
    # save as netcdf
    wps.to_dataset(name='wp').to_netcdf(results_path+"/windpower_TX_ERA5_GWA_" + year1 + "-" + year2 + ".nc")

calculating ERA5 TX GWA 2020-2021
[########################################] | 100% Completed | 10.2s
[########################################] | 100% Completed |  9.7s


In [5]:
del(wps,wind,wind1,alpha,alpha1)

In [6]:
# load wind power generation
wp_loc = xr.open_mfdataset(results_path+"/windpower_TX_ERA5_GWA_????-????.nc")
# aggregate wind power generation to TX
wpTX = wp_loc.sum('location').to_dataframe()
# aggregate wind power generation daily
wpTXd = wpTX.resample('d').sum()
# calculate capacity factors
cfTX = wpTX/turbine_data_era_gwa.capacity.sum()
cfTXd = wpTXd/(turbine_data_era_gwa.capacity.sum()*24)

In [22]:
# save
wpTX.to_csv(results_path + 'wpTXh.csv')
wpTXd.to_csv(results_path + 'wpTXd.csv')
cfTX.to_csv(results_path + 'cfTXh.csv')
cfTXd.to_csv(results_path + 'cfTXd.csv')